In [1]:
import pandas as pd
import os
import numpy as np
from typing import List
from typing import Union
#from pandas_profiling import ProfileReport

pd.options.display.max_rows = 20
pd.options.display.max_columns = None

In [4]:
#test 22/11/2020
def test(a = Union[2, 3]):
    return print(a)
print(2)

TypeError: Union[arg, ...]: each arg must be a type. Got 2.

In [ ]:
df = pd.read_csv(os.path.dirname(os.getcwd())+r"\data"+"\clean_dataset.csv")
SOURCE_PATH = os.path.dirname(os.getcwd())+r"\data"+"\clean_dataset.csv"
COLUMNS_NAN_REPLACE = ["terrace_area", "garden_area", "land_surface", "price", "facades_number"]
COLUMNS_OUTLIERS_IGNORE = ["terrace_area", "source", "garden_area", "postcode", "facades_number"]

filepath: str = SOURCE_PATH
columns_nan_to_replace: List[str] = COLUMNS_NAN_REPLACE
columns_outliers_ignore: List[str] = COLUMNS_OUTLIERS_IGNORE

df = pd.read_csv(filepath)
#deep copy to avoid warnings when indexing
df_cleaned = df.copy(deep=True)
df_cleaned.shape

In [ ]:
profile = df.profile_report()
profile.to_file(os.getcwd()+"\\reports"+"\\df_before_cleaning.html")

replace nan with 0 when relevant

In [ ]:
columns_nan_replaced = []
for c in df_cleaned.columns:
    c_na_count = df_cleaned[c].isna().sum()
    if c_na_count > 0:
        columns_nan_replaced.append(c)
print(columns_nan_replaced)

In [ ]:
df_cleaned = df_cleaned.fillna(0)
for c in df_cleaned.columns:
    c_na_count = df_cleaned[c].isna().sum()
    if c_na_count > 0:
        print(c)

remove if terrace/garden true but area is 0

In [ ]:
df_cleaned[(df_cleaned.terrace==True) & (df_cleaned.terrace_area==0)]

In [ ]:
df_cleaned.building_state_agg.unique()


In [ ]:
print(f'Intial rows {len(df)}')
df_cleaned = df_cleaned[~(((df_cleaned.terrace==True) & (df_cleaned.terrace_area==0))|((df_cleaned.garden==True) & (df_cleaned.garden_area==0)))]
print(f'% kept rows {len(df_cleaned)/len(df)*100}')

Calculating Tukey fences to deal with founded non-normal distributions
NOTE FOR GROUP: effective python way to check for how normal functions are? Maybe already in profiling report?

In [ ]:
def describe_with_tukey_fences(dataframe: pd.DataFrame):
    dataframe_desc = dataframe.describe(percentiles=[0.95, 0.94, 0.75, 0.5, 0.25, 0.06, 0.05], include=np.number)
    dataframe_index = dataframe_desc.index.to_list()
    fence_tukey_min =[dataframe_desc.loc["25%",c] - 1.5 * (dataframe_desc.loc["75%",c] - dataframe_desc.loc["25%",c]) for c in  dataframe_desc.columns]
    fence_tukey_max =[dataframe_desc.loc["75%",c] + 1.5 * (dataframe_desc.loc["75%",c] - dataframe_desc.loc["25%",c]) for c in  dataframe_desc.columns]
    dataframe_desc = dataframe_desc.append(dict(zip(dataframe_desc.columns, fence_tukey_min)), ignore_index=True)
    dataframe_desc = dataframe_desc.append(dict(zip(dataframe_desc.columns, fence_tukey_max)), ignore_index=True)
    dataframe_index.append('fence_tukey_min')
    dataframe_index.append('fence_tukey_max')
    dataframe_desc.index = dataframe_index    
    return dataframe_desc
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

In [ ]:
print(f'Intial rows {len(df)}')
for column in df_desc:
    if column not in COLUMNS_OUTLIERS_IGNORE:
        #fence_tukey_min, fence_tukey_max = df_desc.loc[["fence_tukey_min", "fence_tukey_max"], column]
        fence_tukey_min = df_desc.loc["fence_tukey_min", column]
        fence_tukey_max = df_desc.loc["fence_tukey_max", column]
        p95 = df_desc.loc["95%", column]
        p94 = df_desc.loc["94%", column]
        p06 = df_desc.loc["6%", column]
        p05 = df_desc.loc["5%", column]
        #printing
        row_start = len(df_cleaned)
        if fence_tukey_max >= p95:
            df_cleaned= df_cleaned[df_cleaned[column]<fence_tukey_max]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} Tukey's max outliers in {column}")
        elif (fence_tukey_max < p95) and  (p95 != p94):
            df_cleaned= df_cleaned[df_cleaned[column]<p95]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} percentile max outliers in {column}")
        row_start = len(df_cleaned)
        if fence_tukey_min <= p05:
            df_cleaned= df_cleaned[df_cleaned[column]>fence_tukey_min]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} Tukey's min outliers in {column}")
        elif (fence_tukey_max < p05) and  (p05 != p06):
            df_cleaned= df_cleaned[df_cleaned[column]<p95]
            if len(df_cleaned) < row_start:
                print(f"Excluding {(1-len(df_cleaned)/row_start)*100} percentile min outliers in {column}")
print(f"Excluding {(1-len(df_cleaned)/len(df))*100}")

In [ ]:
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

### Converting categorical variables and adding numerical variables
Categorical variables were analysed by assigning to them median price (target variable).

For postcode a limited number of values remained for some postcodes after removing outliers.

In [ ]:
postcode_price = df_cleaned.groupby('postcode')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per postcode
print(postcode_price.sort_values(by="median", ascending=False).head(5))
print(postcode_price.sort_values(by="mean", ascending=False).head(5))

def get_postcode_price(x):
    postcode_price_median = postcode_price.loc[x,"median"]
    return postcode_price_median
df_cleaned["postcode_median_price"] = df_cleaned["postcode"].apply(get_postcode_price)
df_desc = describe_with_tukey_fences(df_cleaned)
df_desc

For building state more values were available, analysing median price

In [ ]:
state_price = df_cleaned.groupby('building_state_agg')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(state_price.sort_values(by="median", ascending=False).head(5))
print(state_price.sort_values(by="mean", ascending=False).head(5))

def get_building_state_price(x):
    state_price_median = state_price.loc[x,"median"]
    return state_price_median

df_cleaned["building_state_median_price"] = df_cleaned["building_state_agg"].apply(get_building_state_price)
describe_with_tukey_fences(df_cleaned)

For subtypes more values were available too.

In [ ]:
property_subtype_price = df_cleaned.groupby('property_subtype')['price'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(property_subtype_price.sort_values(by="median", ascending=False).head(5))
print(property_subtype_price.sort_values(by="mean", ascending=False).head(5))

def get_property_subtype_price(x):
    subtype_price_median = property_subtype_price.loc[x,"median"]
    return subtype_price_median

df_cleaned["property_subtype_median_price"] = df_cleaned["property_subtype"].apply(get_property_subtype_price)

replacing categorical values of state with facades

In [ ]:
df_cleaned[df_cleaned.property_subtype=='APARTMENT']

In [ ]:
df_with_facades = df_cleaned[df_cleaned.facades_number > 0]

property_subtype_facades = df_with_facades.groupby('property_subtype')['facades_number'].agg([min,max,np.mean,np.median,len])
#verifying count and values per status
print(property_subtype_facades)
print(property_subtype_facades.sort_values(by="median", ascending=False).head(5))
print(property_subtype_facades.sort_values(by="mean", ascending=False).head(5))

def get_property_subtype_facades(x):
    try:
        property_subtype_facades_median = property_subtype_facades.loc[x,"median"]
    except:
        property_subtype_facades_median = None
        #print(x) #error with apartment_block
    return property_subtype_facades_median

df_cleaned["property_subtype_median_facades"] = df_cleaned["property_subtype"].apply(get_property_subtype_facades)

In [ ]:
df_cleaned.shape

In [ ]:
df_cleaned.to_csv(os.getcwd()+"\\outputs"+"\\df_after_cleaning.csv")

In [ ]:
df_new = pd.read_csv(os.getcwd()+"\\outputs"+"\\df_after_cleaning.csv")
df_new.shape

In [ ]:
profile = df_new.profile_report()
profile.to_file(os.getcwd()+"\\reports"+"\\df_after_cleaning.html")
